In [12]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# Loading Data

In [18]:
processed_data = pd.read_csv(r'C:\Projects\a_health_data_analysis\firebase-health-data-analysis\data\2_processed_data.csv')
processed_data.head()

,date,ID,create_time,update_time,sleep_duration_hours,sleep_quality,sleep_interruptions,activity_sedentary_hours,activity_active_minutes,activity_steps,...,vitals_bp_1,vitals_bp_2,vitals_bp_3,vitals_bp_4,vitals_bp_5,vitals_temp_1,vitals_temp_2,vitals_temp_3,vitals_temp_4,vitals_temp_5
0,21/03/2025,1aNK0IlBlOGhWp0JIPGo,2025-03-27T11:43:39.925877Z,2025-03-27T11:43:39.925877Z,6.8,fair,2,9.7,38,5200,...,119/80,121/82,118/79,120/81,122/83,98.6,98.4,98.5,98.7,98.6
1,21/03/2025,2DCJ0E7x9uSiMc8M9Hje,2025-03-27T11:43:39.925877Z,2025-03-27T11:43:39.925877Z,6.8,fair,2,9.7,38,5200,...,119/80,121/82,118/79,120/81,122/83,98.6,98.4,98.5,98.7,98.6
2,17/03/2025,3xP5bsc13fVaehGMH2wd,2025-03-27T11:43:39.925877Z,2025-03-27T11:43:39.925877Z,6.5,poor,3,10.5,35,4500,...,120/80,122/82,118/79,121/81,125/83,98.6,98.4,98.7,98.5,98.8
3,28/03/2025,6r6bGwNOMEfTVRXMO9BR,2025-03-27T11:43:39.925877Z,2025-03-27T11:43:39.925877Z,6.0,poor,3,10.8,35,4800,...,125/82,127/84,124/81,126/83,128/85,98.9,98.7,99.0,98.8,99.1
4,19/03/2025,9A3FURfOO7c55asePIFn,2025-03-27T11:43:39.925877Z,2025-03-27T11:43:39.925877Z,8.1,good,1,8.2,55,8200,...,115/76,117/78,114/75,116/77,118/79,98.4,98.2,98.3,98.5,98.4


In [19]:
cleaned_data = pd.read_csv(r'C:\Projects\a_health_data_analysis\firebase-health-data-analysis\data\cleaned_data.csv')
cleaned_data.head()

,sleep_duration_hours,sleep_quality,sleep_interruptions,activity_sedentary_hours,activity_active_minutes,activity_steps,nutrition_calories,nutrition_water_oz,nutrition_macro_fat_g,nutrition_macro_carbs_g,nutrition_macro_protein_g,avg_heart_rate,avg_temperature,avg_systolic_bp,avg_diastolic_bp,avg_bp
0,6.8,fair,2,9.7,38,5200,2100,48,58,270,90,72.00,98.56,120.0,81.0,120.0/81.0
1,6.8,fair,2,9.7,38,5200,2100,48,58,270,90,72.00,98.56,120.0,81.0,120.0/81.0
2,6.5,poor,3,10.5,35,4500,2200,45,60,290,85,72.17,98.60,121.2,81.0,121.2/81.0
3,6.0,poor,3,10.8,35,4800,2400,45,70,330,85,75.50,98.90,126.0,83.0,126.0/83.0
4,8.1,good,1,8.2,55,8200,1850,60,50,220,100,66.17,98.36,116.0,77.0,116.0/77.0


# Data Preparation

In [15]:
# Display basic information about the dataset
print("Dataset Shape:", cleaned_data.shape)
print("\nColumn Types:")
print(cleaned_data.dtypes)

Dataset Shape: (20, 16)

Column Types:
sleep_duration_hours         float64
sleep_quality                 object
sleep_interruptions            int64
activity_sedentary_hours     float64
activity_active_minutes        int64
activity_steps                 int64
nutrition_calories             int64
nutrition_water_oz             int64
nutrition_macro_fat_g          int64
nutrition_macro_carbs_g        int64
nutrition_macro_protein_g      int64
avg_heart_rate               float64
avg_temperature              float64
avg_systolic_bp              float64
avg_diastolic_bp             float64
avg_bp                        object
dtype: object


In [16]:
# Check for missing values
print("\nMissing Values:")
print(cleaned_data.isnull().sum())


Missing Values:
sleep_duration_hours         0
sleep_quality                0
sleep_interruptions          0
activity_sedentary_hours     0
activity_active_minutes      0
activity_steps               0
nutrition_calories           0
nutrition_water_oz           0
nutrition_macro_fat_g        0
nutrition_macro_carbs_g      0
nutrition_macro_protein_g    0
avg_heart_rate               0
avg_temperature              0
avg_systolic_bp              0
avg_diastolic_bp             0
avg_bp                       0
dtype: int64


## Data Labeling

In [17]:
def classify_health(row):
    # Poor Health Conditions
    if (row['sleep_duration_hours'] < 6.5 or  
        row['sleep_quality'] == 'poor' or   
        row['sleep_interruptions'] >= 3 or
        row['avg_heart_rate'] > 80 or         
        row['avg_systolic_bp'] > 130 or       
        row['avg_diastolic_bp'] > 85 or      
        row['activity_steps'] < 5000):         
        return 'Poor'

    # Good Health Conditions
    elif (row['sleep_duration_hours'] > 8 and  
          row['sleep_quality'] in ['good', 'excellent'] and  
          row['sleep_interruptions'] <= 1 and
          row['avg_heart_rate'] < 70 and             
          row['avg_systolic_bp'] < 120 and    
          row['avg_diastolic_bp'] < 80 and   
          row['activity_steps'] > 8000):         
        return 'Good'

    else:
        return 'Moderate'

## One Hot Encoding

In [ ]:
#One Hot Encoding

## Scaling

## Train and Test Split

# Training Models